<a href="https://colab.research.google.com/github/cedro3/Tune-A-Video/blob/main/Tune_A_Video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title **Setup**
# check GPU
! nvidia-smi -L

# change torch version
! pip install torch==1.13.0 torchvision==0.14.0

# install xformers
! pip install -q https://github.com/brian6091/xformers-wheels/releases/download/0.0.15.dev0%2B4c06c79/xformers-0.0.15.dev0+4c06c79.d20221205-cp38-cp38-linux_x86_64.whl

# copy github code & install library
! git clone https://github.com/cedro3/Tune-A-Video.git
%cd Tune-A-Video
! pip install -r requirements.txt

# download model
! mkdir checkpoints 
%cd checkpoints
!git lfs install
!git clone https://huggingface.co/CompVis/stable-diffusion-v1-4
%cd ..

In [ ]:
#@title **Train**
! accelerate launch train_tuneavideo.py --config="configs/man-surfing.yaml"

In [ ]:
#@title **Make GIF**
from tuneavideo.pipelines.pipeline_tuneavideo import TuneAVideoPipeline
from tuneavideo.models.unet import UNet3DConditionModel
from tuneavideo.util import save_videos_grid
import torch

id_name = "2023-01-31T12-15-52" #@param {type:"string"}
model_id = "./outputs/man-surfing_lr3e-5_seed33/" + id_name
unet = UNet3DConditionModel.from_pretrained(model_id, subfolder='unet', torch_dtype=torch.float16).to('cuda')
pipe = TuneAVideoPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", unet=unet, torch_dtype=torch.float16).to("cuda")

prompt = "a boy, wearing a birthday hat, is surfing" #@param ["a panda is surfing", "a boy, wearing a birthday hat, is surfing", "a raccoon is surfing, cartoon style", "Iron Man is surfing in the desert"]
video = pipe(prompt, video_length=8, height=512, width=512, num_inference_steps=50, guidance_scale=7.5).videos

# save video
save_videos_grid(video, "outputs/video.gif")


In [ ]:
#@title **Play GIF**
from IPython.display import Image
Image("outputs/video.gif", format='png')

In [ ]:
#@title **Download GIF** ( for chrome only)
from google.colab import files
files.download("outputs/video.gif")